# Project 3: Perfect Vine Compilation
## Group 7
### Hayley Carney, Rohan Enagala, Stephen Jennings, Basil Mullings, Utkarsh Karki, Sandeep Sonawane

## Imports and Installs

In [1]:
#TODO Add in import statements as needed
import json
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import time
from pathlib import Path
from PIL import Image

In [14]:
import pandas as pd
def display_video_thumbnails(df_or_ids, max_images=25, columns=5, title="Selected Videos"):
    """
    Retrieves and displays thumbnail images for a list of videos.

    Args:
        df_or_ids (pd.DataFrame or list/pd.Series): 
            Either a DataFrame containing 'video_id_short' or a list/Series 
            of 'video_id_short' strings.
        max_images (int): The maximum number of images to display. Defaults to 25.
        columns (int): The number of columns in the display grid. Defaults to 5.
        title (str): The title for the plot. Defaults to "Selected Videos".
    """
    
    # --- 1. Determine the list of short IDs ---
    if isinstance(df_or_ids, pd.DataFrame):
        if 'video_id_short' not in df_or_ids.columns:
            print("❌ Error: DataFrame must contain the 'video_id_short' column.")
            return
        # Use the IDs from the DataFrame (e.g., if you pass df_data[:10])
        short_ids = df_or_ids['video_id_short'].tolist()
        
    elif isinstance(df_or_ids, (list, pd.Series)):
        # Use the list or Series directly (e.g., if you pass short_ids_list)
        short_ids = list(df_or_ids) 
        
    else:
        print("❌ Error: Input must be a DataFrame, list, or pandas Series of IDs.")
        return

    if not short_ids:
        print("📦 Found 0 IDs to display.")
        return


    num_found = len(short_ids)
    print(f"📦 Found {num_found} IDs. Displaying first {min(num_found, max_images)}.")
    
    # Limit the number of IDs to display
    ids_to_display = short_ids[:max_images]
    num_to_display = len(ids_to_display)
    
    # Calculate rows needed for the grid display
    rows = (num_to_display + columns - 1) // columns
    
    # 2. Setup the plot grid
    plt.figure(figsize=(columns * 3, rows * 3))
    plt.suptitle(f"Thumbnails for: {title}", fontsize=16)
    # print(",".join(short_ids)) # Optionally removed this large print

    # 3. Iterate through IDs and display images
    for i, video_id in enumerate(ids_to_display):
        # Construct the full path
        image_path = THUMBNAIL_BASE_DIR / f"{video_id}.png"
        
        plt.subplot(rows, columns, i + 1)
        
        # Determine the ID string to display in the title
        displayed_id = video_id            
        try:
            # Load and display the image
            img = Image.open(image_path)
            plt.imshow(img)

            # Use the possibly censored ID for the title
            plt.title(f"{displayed_id}", fontsize=8) 
            plt.axis('off') # Hide axis ticks and labels
            
        except FileNotFoundError:
            plt.title("Missing", fontsize=8, color='red')
            plt.text(0.5, 0.5, "Image Not Found", ha='center', va='center', color='red')
            plt.axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout
    plt.show()

In [ ]:
def load_embeddings(jsonl_path):
    """
    Loads embeddings, filenames, and video_id_short from the JSONL file, 
    ensures consistent shape, and mean-pools any (16, D) time-series 
    embeddings to (D,).
    
    Returns: (DataFrame with 'filename', 'embedding', 'video_id_short', 
              list of numpy arrays)
    """
    data = []
    skipped_count = 0
    expected_shape = None
    print(f"Loading data from {jsonl_path}...")
    
    raw_data = []
    try:
        with open(jsonl_path, 'r') as f:
            for line in f:
                if line.strip():
                    item = json.loads(line)
                    raw_data.append({
                        'filename': item['filename'],
                        # 🔑 FIX: Extract the video_id_short field
                        'video_id_short': item.get('video_id_short', None),
                        'whisper_transcript': item.get('whisper_transcript', None),
                        'embedding': np.array(item['embedding'], dtype=np.float32)
                    })
    except (FileNotFoundError, json.JSONDecodeError) as e:
        print(f"Error during file read: {e}")
        return pd.DataFrame(), []

    if not raw_data:
        print("No data found in the input file.")
        return pd.DataFrame(), []

    for item in raw_data:
        current_embedding = item['embedding']
        # filename = item['filename'] # Not needed here, but kept in item
        
        if current_embedding.size == 0:
            skipped_count += 1
            continue

        if current_embedding.ndim == 2 and current_embedding.shape[0] == 16:
            current_embedding = np.mean(current_embedding, axis=0)
            item['embedding'] = current_embedding

        if expected_shape is None:
            expected_shape = current_embedding.shape
            data.append(item)
        elif current_embedding.shape == expected_shape:
            data.append(item)
        else:
            skipped_count += 1
            
    # Note: df now includes 'video_id_short'
    df = pd.DataFrame(data)
    embeddings = list(df['embedding'].values)

    print(f"Successfully loaded {len(embeddings)} consistent embeddings (Skipped {skipped_count} invalid/inconsistent entries).")
    return df, embeddings

## Data Read In

In [12]:
#TODO Add data to repo for easy read in
THUMBNAIL_BASE_DIR = Path('/Users/hayleygoldblatt/Documents/cosc545/m7_p3_perfect_vine_compilation/Data/thumbs')
INPUT_JSONL_FILE = '/Users/hayleygoldblatt/Documents/cosc545/m7_p3_perfect_vine_compilation/Data/after_last_cleaning_all_embeddings.jsonl'
N_CLUSTERS = 5000
RANDOM_STATE = 41
FIG_SIZE = (12, 10)
DOT_SIZE = 1

In [13]:
df_data, embeddings_list = load_embeddings(INPUT_JSONL_FILE)

Loading data from /Users/hayleygoldblatt/Documents/cosc545/m7_p3_perfect_vine_compilation/Data/after_last_cleaning_all_embeddings.jsonl...
Successfully loaded 28385 consistent embeddings (Skipped 0 invalid/inconsistent entries).


In [15]:
df_data.head()

,filename,video_id_short,whisper_transcript,embedding
0,20130626125209_182352F6-CFF3-497C-9891-8FF7D18...,182352F6-C,All DOGROUND... ALL DOGROUND... Call ohmd,"[-0.031610012, 0.03581158, 0.031906445, 0.0145..."
1,20130627194331_18C3F6CB-6E8C-4DD1-A245-CCA4CF2...,18C3F6CB-6,"Baby, Baby!","[-0.02479268, 0.029688096, 0.028634664, 0.0226..."
2,20130628013622_14CF3EAD-D4F8-4174-885A-27A2D49...,14CF3EAD-D,How are you? Great Bob! We're all in a little ...,"[-0.015824113, 0.033187184, 0.01681333, 0.0065..."
3,20130628211747_18112C4B-742F-4745-ACB1-C76F89A...,18112C4B-7,,"[-0.024460625, 0.03128573, 0.0068606776, 0.033..."
4,20130702110757_0AC9933B-F979-4A43-BB29-89C0C05...,0AC9933B-F,None,"[-0.026678782, 0.03043931, 0.0365816, 0.024619..."


## Data Preprocessing
https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932

In [18]:
#TODO remove vines with swears/inappropriate content
from profanity_check import predict
df_data['profanity_score'] = predict(df_data['whisper_transcript'].fillna(''))
df_data = df_data[df_data['profanity_score'] == 0].reset_index(drop=True)
df_data.shape

/opt/anaconda3/envs/cosc545/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/cosc545/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.7.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/cosc545/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.7.2 when using ve

(25860, 5)

In [ ]:
#TODO filter to niche topics

## Clustering Analysis
https://medium.com/@RobuRishabh/clustering-text-data-with-k-means-and-visualizing-with-t-sne-9bc1fe7d8fed

In [25]:
# Expand the 'Features' column into new columns
expanded_features = df_data['embedding'].apply(pd.Series)

# Optionally, rename the new columns for clarity
expanded_features.columns = [f'embedding_{i+1}' for i in range(expanded_features.shape[1])]

# Concatenate the expanded features with the original DataFrame (excluding the original list column)
df_expanded = pd.concat([expanded_features], axis=1)


In [26]:
df_expanded.head()

,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,...,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767,embedding_768
0,-0.031610,0.035812,0.031906,0.014598,-0.037266,0.023161,-0.002574,-0.031405,0.012062,-0.055055,...,-0.024395,0.019522,0.049779,-0.027164,-0.020035,-0.095322,-0.004795,0.015109,0.012812,0.036391
1,-0.024793,0.029688,0.028635,0.022613,-0.040009,0.026508,-0.002295,-0.036946,0.018670,-0.050425,...,-0.026428,0.027089,0.050627,-0.026537,-0.019204,-0.097297,-0.001054,0.014319,0.007215,0.031497
2,-0.015824,0.033187,0.016813,0.006594,-0.023289,0.032709,-0.022123,-0.035444,0.014615,-0.044755,...,-0.041138,0.017483,0.061825,-0.018192,-0.021996,-0.082239,-0.009018,0.034478,0.018713,0.033017
3,-0.024461,0.031286,0.006861,0.033299,-0.037236,0.015712,0.001993,-0.025504,0.020557,-0.051506,...,-0.024206,0.040424,0.035380,-0.027134,-0.021946,-0.101082,0.002300,0.028146,0.000513,0.034151
4,-0.026679,0.030439,0.036582,0.024620,-0.039549,0.024891,0.000604,-0.035731,0.015495,-0.054270,...,-0.020595,0.026193,0.048427,-0.030858,-0.024733,-0.101732,-0.000976,0.010061,0.003600,0.031207


In [28]:
#TODO Test K-Means to determine number of clusters (elbow plot)

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


# 2. Calculate WCSS (Within-Cluster Sum of Squares) for different k values
wcss = []
k_values = range(1, 100)  # Test k from 1 to 10

for k in k_values:
    kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df_expanded)
    wcss.append(kmeans.inertia_) # inertia_ is the WCSS

# 3. Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(k_values, wcss, marker='o', linestyle='--')
plt.title('Elbow Plot for Optimal K')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.grid(True)
plt.show()


KeyboardInterrupt: 

In [ ]:
#TODO perform K-Means with optimal number of clusters

In [ ]:
#TODO compare K-Means results with DBSCAN results

In [ ]:
#TODO Visualize K-Means clusters with t-SNE

## Vine Selection

In [ ]:
#TODO Based on clustering results, create compilations of similar vines

## Compilation Creation

## Reflection Section